In [1]:
import warnings
warnings.filterwarnings("ignore")

from data_loading import *

from xai_agg.agg_exp import *
from xai_agg.utils import *

import pandas as pd
import numpy as np

from scipy.stats import pearsonr, spearmanr, kendalltau

2025-01-15 20:17:02.980828: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-15 20:17:03.004642: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
dataset_name, preprocessed_data, categorical_features, X, y, X_train, X_test, y_train, y_test, clf = load_german_credit_data()

Accuracy: 0.7696335078534031
ROC AUC: 0.6830357142857143


In [3]:
evaluator = ExplanationModelEvaluator(clf, X_train, categorical_features)
evaluator.init()

Epoch 1/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.3095 - val_loss: 1.2573
Epoch 2/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2597 - val_loss: 1.2390
Epoch 3/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2048 - val_loss: 1.2219
Epoch 4/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2370 - val_loss: 1.2051
Epoch 5/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2388 - val_loss: 1.1888
Epoch 6/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1843 - val_loss: 1.1725
Epoch 7/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1812 - val_loss: 1.1558
Epoch 8/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1663 - val_loss: 1.1389
Epoch 9/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1222 - val_loss: 1.1224
Epoch 10/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1263 - val_loss: 1.1063
Epoch 11/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.0929 - val_loss: 1.0901
Epoch 12/500
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1

### Misc

In [4]:
row = X_test.iloc[0]

evaluator.faithfullness_correlation(LimeWrapper, row, len_subset=1, iterations=1)

nan

# Generating metric vectors and calculating correlation between them

In [5]:
# Getting a few reanknings
exps = [LimeWrapper(clf, X_train, categorical_features), ShapTabularTreeWrapper(clf, X_train, categorical_features), AnchorWrapper(clf, X_train, categorical_features)]

indexes = np.random.choice(X_test.index, 100, replace=False)


In [8]:
import concurrent.futures

comp_arr = [] # original, new1, new2, ...

def process_index(idx):
    local_comp_arr = []
    for exp in exps:
        row = []
        explanation = exp.explain_instance(X_test.loc[idx])
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=False, len_subset=1, iterations=100))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=False, len_subset=1, iterations=100))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="sum", len_subset=1, iterations=100))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="percentile", len_subset=1, iterations=100))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="avg", len_subset=1, iterations=100))
        row.append(evaluator.faithfullness_correlation(exp, X_test.loc[idx], explanation=explanation, rank_based=True, rb_alg="inverse", len_subset=1, iterations=100))
        
        # row.append(evaluator.complexity(exp, X_test.loc[idx], explanation=explanation))
        # row.append(evaluator.nrc(exp, X_test.loc[idx], explanation=explanation))
        
        local_comp_arr.append(row)
    return local_comp_arr
    
with concurrent.futures.ProcessPoolExecutor(5) as executor:
    results = list(executor.map(process_index, indexes))

for result in results:
    comp_arr.extend(result)

Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible result. The desired precision threshold might not be achieved due to the quantile-based discretisation of the numerical features. The resolution of the bins may be too large to find an anchor of required precision. Consider increasing the number of bins in `disc_perc`, but note that for some numerical distribution (e.g. skewed distribution) it may not help.
Could not find an anchor satisfying the 0.95 precision constraint. Now ret

In [10]:
comp_arr

array([[0.71461734, 0.55524511,        nan,        nan,        nan,
               nan],
       [0.45510969, 0.48920958,        nan,        nan,        nan,
               nan],
       [0.63910865, 0.799729  ,        nan,        nan,        nan,
               nan],
       ...,
       [0.5432679 , 0.6295685 ,        nan,        nan,        nan,
               nan],
       [0.63375906, 0.65377019,        nan,        nan,        nan,
               nan],
       [0.72954002, 0.66152614,        nan,        nan,        nan,
               nan]])

In [9]:
from scipy.stats import spearmanr

comp_arr = np.array(comp_arr)
rank_algs = ["Itself", "sum", "percentile", "avg", "inverse"]
# rank_algs = ["NRC"]

pearson_corr = [np.corrcoef(comp_arr[:, 0], comp_arr[:, i])[0, 1] for i in range(1, comp_arr.shape[1])]
spearman_corr = [spearmanr(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]
kendall_corr = [kendalltau(comp_arr[:, 0], comp_arr[:, i]).correlation for i in range(1, comp_arr.shape[1])]

correlation_df = pd.DataFrame({
    'Rank Algorithm': rank_algs,
    'Pearson Correlation': pearson_corr,
    'Spearman Correlation': spearman_corr,
    'Kendall Correlation': kendall_corr
})

print(dataset_name)
display(correlation_df)

GERMAN


,Rank Algorithm,Pearson Correlation,Spearman Correlation,Kendall Correlation
0,Itself,0.737277,0.7069,0.521561
1,sum,NaN,NaN,NaN
2,percentile,NaN,NaN,NaN
3,avg,NaN,NaN,NaN
4,inverse,NaN,NaN,NaN
